In [2]:
import pandas as pd
# Import
from pandarallel import pandarallel

# Initialization
pandarallel.initialize()

import os
import json
# !pip install pywer
import pywer
# !pip install pyjarowinkler
from pyjarowinkler import distance as jwdistance
from tqdm import tqdm

class Const:
    OCR = 'ocr'
    GROUND = 'groundtruth'
    REGION = 'region'
    LINE = 'line'
    SENTENCE = 'sentence'
    FILE = 'filename'
    DATASET = 'dataset_name'
    PREDICTION = 'prediction'
    PROMPT = 'prompt'
    LANGUAGE = 'language'
    NONE = None


INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
# !pip install pandarallel
# !pip install pywer
# !pip install pyjarowinkler

### Lookup datasets

In [4]:
datasets = []

for type_document in ['ocr', 'htr', 'asr']:
    for root, dirs, files in os.walk(f'../data/datasets/{type_document}/converted'):
        for file in files:
            if file.endswith(".jsonl"):
                input_file = os.path.join(root, file)
                if 'sample' not in input_file:
                    with open(input_file) as f:
                        lines = f.read().splitlines()
                    df_inter = pd.DataFrame(lines)
                    df_inter.columns = ['json_element']
                    df_inter['json_element'].apply(json.loads)
                    df = pd.json_normalize(df_inter['json_element'].apply(json.loads))

                    dataset_name = root.split('/')[-1].replace('_', '-')
                    print(input_file, dataset_name)
                    df['dataset_name'] = [dataset_name] * len(df)
                    if 'ajmc' in dataset_name:
                        df['language'] = ['el'] * len(df)
                    if 'overproof' in dataset_name:
                        df['language'] = ['en'] * len(df)
                    if 'impresso' in dataset_name:
                        df['language'] = ['de'] * len(df)

                    datasets.append(df)
            

../data/datasets/ocr/converted/icdar-2019/icdar-2019.jsonl icdar-2019
../data/datasets/ocr/converted/ajmc-primary/ajmc_primary_text.jsonl ajmc-primary
../data/datasets/ocr/converted/overproof/overproof.jsonl overproof
../data/datasets/ocr/converted/ajmc-mixed/ajmc_mixed.jsonl ajmc-mixed
../data/datasets/ocr/converted/impresso/impresso-nzz.jsonl impresso
../data/datasets/ocr/converted/icdar-2017/icdar-2017.jsonl icdar-2017
../data/datasets/htr/converted/htrec/htrec.jsonl htrec
../data/datasets/asr/converted/ina/ina.jsonl ina


In [5]:
print('Number of unique lines/sentences/regions.\n')
for dataset in datasets:
    print('Dataset:', dataset['dataset_name'].unique()[0], len(dataset), 'with duplicates')
    print('No. lines:', dataset['ocr.line']. nunique(), '/', len(dataset['ocr.sentence']), 
          'No. sentences:', dataset['ocr.sentence']. nunique(), '/', len(dataset['ocr.sentence']), 
          'No. regions:', dataset['ocr.region']. nunique(), '/', len(dataset['ocr.region']))
    print('-'*80)

Number of unique lines/sentences/regions.

Dataset: icdar-2019 404 with duplicates
No. lines: 0 / 404 No. sentences: 404 / 404 No. regions: 41 / 404
--------------------------------------------------------------------------------
Dataset: ajmc-primary 57 with duplicates
No. lines: 40 / 57 No. sentences: 27 / 57 No. regions: 9 / 57
--------------------------------------------------------------------------------
Dataset: overproof 2669 with duplicates
No. lines: 2278 / 2669 No. sentences: 399 / 2669 No. regions: 41 / 2669
--------------------------------------------------------------------------------
Dataset: ajmc-mixed 1291 with duplicates
No. lines: 535 / 1291 No. sentences: 379 / 1291 No. regions: 33 / 1291
--------------------------------------------------------------------------------
Dataset: impresso 1563 with duplicates
No. lines: 1256 / 1563 No. sentences: 577 / 1563 No. regions: 203 / 1563
--------------------------------------------------------------------------------
Dataset

## Step 1: Loading of preliminary results


In [6]:
def read_line_by_line(filename):
    pass

def json_load(text):
    
    try:
        loaded_line = json.loads(text)
    except Exception as ex:
        print(ex)
        print(text[:30], '...')
        loaded_line = 'No text'
    return loaded_line

In [7]:
def get_icdar_2017_lang(filename):
    lang = filename.split('/')[-2].split('_')[0]
    if lang =='eng':
        lang = 'en'
    return lang

In [8]:
import gc; gc.collect()

0

In [9]:
from tqdm import tqdm
import os

results = []

# First traversal to get count of .jsonl files
jsonl_count = 0
for root, dirs, files in os.walk('../data/output'):
    for file in files:
        if file.endswith(".jsonl"):
            jsonl_count += 1

# Second traversal to do the processing with tqdm progress bar
with tqdm(total=jsonl_count) as pbar:
    for root, dirs, files in os.walk('../data/output'):
        for file in files:
            if file.endswith(".jsonl"):
                input_file = os.path.join(root, file)
                is_few = False
                if 'few' in input_file:
                    prompt = root.split('/')[-2]
                    is_few = True
                else:
                    prompt = root.split('/')[-2]
                    
                if 'sample' not in input_file:
#                     if 'prompt_complex_lang' in input_file:
#                         print(input_file)
#                     prompt = root.split('/')[-2]
                    try:
                        with open(input_file) as f:
                            text = f.read()
                    
                        with open(input_file) as f:
                            lines = f.readlines()
                    except Exception as ex:
                        print('We could not load {} {}'.format(input_file, ex))
                        continue
                    # Check correct lines
                    text = text.replace('\n', '')
                    text_list = text.split('}}{"')
                    json_objects = []

                    for i, t in enumerate(text_list):
                        if i != 0:
                            t = '{"' + t
                        if i != len(text_list) - 1:
                            t = t + '}'
                        if not t.endswith('}}'):
                            json_objects.append(t + '}\n')
                        else:
                            json_objects.append(t + '\n')
                        
                    df_inter = pd.DataFrame(json_objects)
                    df_inter.columns = ['json_element']

                    dataset_name = root.split('/')[-1].replace('_', '-')
                    model_dataset_name = file[8:-6]
                    model_name = model_dataset_name.replace(root.split('/')[-1] + '-', '').strip()
                    try:
                        df_inter['json_element'].apply(lambda x: json_load(x))
                        df = pd.json_normalize(df_inter['json_element'].apply(json.loads))

                        df['model'] = [model_name] * len(df)

                        df['dataset_name'] = [dataset_name] * len(df)
                        df['prompt'] = [prompt] * len(df)
                        try:
                            with open(f'../data/prompts/{prompt}.txt', 'r') as f:
                                prompt_text = f.read()
                            prompt_text = prompt
                        except:
                            prompt_text = 'prompt_complex_03_per_lang'
                            
                        df['prompt_text'] = [prompt_text] * len(df)
                        
                        if is_few:
                            df['type'] = ['few-shot'] * len(df)
                        else:
                            df['type'] = ['zero-shot'] * len(df)

                        df['dataset_name'] = [dataset_name] * len(df)
                        
                        if 'ajmc' in dataset_name:
                            df['language'] = ['el'] * len(df)
                        if 'ina' in dataset_name:
                            df['language'] = ['fr'] * len(df)
                        if 'overproof' in dataset_name:
                            df['language'] = ['en'] * len(df)
                        if 'impresso' in dataset_name:
                            df['language'] = ['de'] * len(df)
                        if 'htrec' in dataset_name:
                            df['language'] = ['el'] * len(df)

#                         print(dataset_name, model_name, prompt)

                        if dataset_name == 'icdar-2017':
                            df['language'] = df['filename'].apply(get_icdar_2017_lang)
                        
                        df['file'] = file
                    
                        results.append(df)
#                         print(df.prompt.unique())
                    except Exception as ex:
                        print('We could not load {} {}'.format(input_file, ex))
                    pbar.update()


 73%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                                 | 816/1114 [02:53<01:11,  4.19it/s]

Expecting value: line 1 column 1 (char 0)
}
 ...
We could not load ../data/output/few_shot/prompt_complex_01/overproof/results-3few-shot-overproof-facebook-opt-6.7b.jsonl Expecting value: line 1 column 1 (char 0)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1114/1114 [03:49<00:00,  4.85it/s]


In [10]:
results[0]['file'][0]

'results-icdar-2019-facebook-opt-6.7b.jsonl'

In [11]:
results[0]['prompt_text'][0]

'prompt_complex_03_per_lang'

In [12]:
results[0]['type'][0]

'zero-shot'

In [13]:
results[0].head()

,language,filename,dataset_name,ocr.line,ocr.sentence,ocr.region,groundtruth.line,groundtruth.sentence,groundtruth.region,prediction.prompt,prediction.sentence,prediction.region,model,prompt,prompt_text,type,file
0,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,"Charles, par la grace de Dieu, etc.","Charles, par la grace de Dieu, etc. Savoir fai...",None,"Charles, par la grace de Dieu, etc.","Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
1,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,"Savoir faisons à touz, presens et avenir, à no...","Charles, par la grace de Dieu, etc. Savoir fai...",None,"Savoir faisons à touz, presens et avenir, à no...","Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
2,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,Et avant que le dit Amelin peust sur ce remedi...,"Charles, par la grace de Dieu, etc. Savoir fai...",None,et avant que le dit Amelin peust sur ce remedi...,"Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
3,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,linceulz et ii.,"Charles, par la grace de Dieu, etc. Savoir fai...",None,linceulz et n,"Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl
4,fr,../../data/datasets/ocr/original/icdar-2019/IC...,icdar-2019,None,"couvertes, et d’ilecques s’en alerent là où bo...","Charles, par la grace de Dieu, etc. Savoir fai...",None,"convertes, et d'ilecques s'en alerent là où bo...","Charles, par la grace de Dieu, etc. Savoir fai...",Veuillez nous aider à réviser et à corriger le...,/s>Veuillez nous aider à réviser et à corriger...,/s>Veuillez nous aider à réviser et à corriger...,facebook-opt-6.7b,prompt_complex_lang,prompt_complex_03_per_lang,zero-shot,results-icdar-2019-facebook-opt-6.7b.jsonl


In [14]:
# !pip install python-levenshtein

In [15]:
import mbleven

## Post-process

In [16]:
import Levenshtein
from Levenshtein import distance
from mbleven import compare
import distance as faster_distance

# def compute_normalized_levenshtein_similarity(ground_truth_text, ocr_text):
#     length = max(len(ocr_text), len(ground_truth_text))
#     levenshtein_distance = distance(ocr_text, ground_truth_text)
#     similarity = (length - levenshtein_distance) / length
#     return similarity

def compute_normalized_levenshtein_similarity(ground_truth_text, ocr_text):
    length = max(len(ocr_text), len(ground_truth_text))
#     levenshtein_distance = faster_distance.levenshtein(ocr_text, ground_truth_text, normalized=True)
    levenshtein_distance = distance(ocr_text, ground_truth_text)
    levenshtein_distance = (length - levenshtein_distance) / length
#     return 1.0 - levenshtein_distance
    return levenshtein_distance

def compute_jaccard(ocr_text, ground_truth_text):
    try: 
        return jwdistance.get_jaro_distance(ocr_text, ground_truth_text)
    except:
        return 0.0

def get_improvement(original_similarity, corrected_similarity):
    
    if original_similarity == 0:
        return min(max(corrected_similarity, -1), 1)
    elif original_similarity != corrected_similarity:
        return min(max((corrected_similarity - original_similarity) / original_similarity, -1), 1)
    elif original_similarity == corrected_similarity:
        return 0
    else:
        return 0 if corrected_similarity < 1 else 1


In [17]:
gt_text = "149 Obrázek z maloměstského kukátka. Podává L. Gro 149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw"
ocr_text = "149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw149 Obrázek z maloměstského kukátka. Podává L. Grw"
pred_text = """Correct the text: "149 Obrázek z maloměstského kukátka. Podává L. Grwsmannová-Brodská. Místo děje: salon paní stavitelky; doba: čas věnovaný kávové „visitě"; jednající osoby: dámy přednější honorace městské, z nichž •většina je mladá a některé skutečně hezké. Na stole prostřeném krásným ubrusem damaškovým stojí talíře s koláčky, věnečky a preclíčky, kol toho pěkně se vyjímají křišťálové sklenky s vodou, stříbrné lžičky a šálky z jemného porcelánu, jejichž vonný obsah na přítomné paničky zdá se velmi blaze působiti. Ze živějšího hovoru vyznívá právě hlas paní notářky, která ve spravedlivém rozhorlení mluví: „Ano, mé dámy, již jest to takové! Samy ráčily jste býti svědky, jak svorně i jednohlasně byl přijat návrh paní sládkové, abychom si pořídily kroje národní a tak přispěly ku zvýšení lesku slavnosti, již pořádá náš statečný studentský spolek „Hvězda", a když již nás páni akademikové poctili důvěrou, že v naše ruce složili starost o buffet a jiné ještě funkce, tož měly bychom snad též jiti za příkladem slečen berních a vzdáti se činnosti jen proto, že se zdá paní berní národní kroj pro tři dcery býti nějakou zby tečnou výlohou?" Paničky projevovaly svoji nevoli, každá jiným spůsobem. Mladá paní adjunktová v duchu si umiňovala, že ve svém přátel ství k berňovům trochu ochladne; to tak! aby ten jejich pošetilý nápad, úóinkovati při slavnosti v obyčejném oděvu, přece zvítězil a dámám se bylo odříci těch půvabných krojů venkovských, co by si jen ona, paní adjunktová, počala s tou haldou brokátu, atlasu, krajek, stuh a aksamitu, za což vydala nejednu desítku, utěšujíc se tím, jak jí to bude slušeti! Hm, a škodu z toho také míti nebude, muž se bude musit po několik měsíců uskrovnit, služka se má beztoho též až příliš dobře, uhradí se to na domácnosti a bude! Nyní ujala se slova paní doktorka: „Aj, od berních to není nic divného, považte jen: tolik dětí! Vždyť my všecky víme, že kdyby sobě slečny toillety samy ne řídily, mnohého by nemohly míti; ony pak mají zásadu: nemá-li být něco pěkné, tož raději nic!" „Pravda, ale slečny Elišky, té nejmladší, jest mně líto; těšila se velice na selský kroj." „Ba ano, byla by v něm vypadala roztomile." „Nyní má po radosti." „Inu, proč má tak nepřející matinku." „To není to, má drahá, jest v tom však jiný háček." „Ah, ano; vždyť víme, že sotva tak tak vyjdou." „Ale na knihy, které jsi tvá, jinak našel byste jen krytí těch mladých mozí, jinak je vydáte." „A tak bude, ostatně jdu až na knihy; ale slyšet jsem od někoho, že o krojech něco dělat, a že to bylo bude dělat národní, a když jsi tento národní učil jak, pak by měl nalézt tuto kartu, ta by se jistě mohla vyměnit se všemi. Můžete-li to od nás odkázat?" Všem ozdravila její hlas, když řekla: „Já, já! a já ho učím. S těmi páne akademiky jsem již mohla vyprávět o světě, v němž vznítí v některé nocy a jednoho dne vyrostá vám hrozný kouzelný strom zůstane, o němž je psáno: ‚Už jen žádat!" 50 V tuto chvíli koupili mohou-li všichni pánové i paní, že když jsi to čerpala, dá se jen kupit. Přižili to, a já jim vám koupi ukáži.
„A já už jste tím vyděštila. Pánové, děkuji vám za chytré vědomí, se kterým vám projeví, že se už točí dívčí zrcadlo." – 151"""
                
get_improvement(compute_normalized_levenshtein_similarity(gt_text, ocr_text), 
                compute_normalized_levenshtein_similarity(gt_text, pred_text))


# -0.7713953488372093

-0.7713953488372093

In [18]:
compute_normalized_levenshtein_similarity(gt_text, ocr_text)

0.9245323586325521

In [19]:
gt_text = "302."
ocr_text = "302."
pred_text = "302."

get_improvement(compute_normalized_levenshtein_similarity(gt_text, ocr_text), 
                compute_normalized_levenshtein_similarity(gt_text, pred_text))



0

In [20]:
gt_text = 'testing'
ocr_text = 'reresting'
pred_text = 'testing'

compute_normalized_levenshtein_similarity(gt_text, ocr_text), distance(ocr_text, gt_text)

(0.6666666666666666, 3)

In [21]:
gt_text = 'testing'
ocr_text = 'rffesting'

compare(ocr_text, gt_text, transpose=True), distance(ocr_text, gt_text), faster_distance.levenshtein(ocr_text, gt_text)

(-1, 3, 3)

In [22]:
gt_text = 'testing'
ocr_text = 'resting'

compute_normalized_levenshtein_similarity(gt_text, ocr_text)

0.8571428571428571

In [23]:
get_improvement(compute_normalized_levenshtein_similarity(gt_text, ocr_text), 
                compute_normalized_levenshtein_similarity(gt_text, pred_text))


0.16666666666666674

In [24]:
import Levenshtein

# Example strings
s1 = "testing"
s2 = "resting"

# Calculate the Levenshtein distance
lev_distance = Levenshtein.distance(s1, s2)
print(f"Levenshtein distance: {lev_distance}")

# Calculate the Levenshtein similarity
similarity = (max(len(s1), len(s2)) - lev_distance) / max(len(s1), len(s2))
print(f"Levenshtein similarity: {similarity:.2f}")


Levenshtein distance: 1
Levenshtein similarity: 0.86


In [25]:
texts_to_remove = ["ΔΙΟΡΘΩΜΈΝΟ ΚΕΊΜΕΝΟ:", "CORRECTED TEXT:", "КОРИГИРАН ТЕКСТ:", "OPRAVENÝ TEXT:", 
                   "KORRIGIERTER TEXT:",
                  "TEXTO CORREGIDO:", "TEXTE CORRIGÉ:", "GECORRIGEERDE TEKST:", "POPRAWIONY TEKST:",
                  "POPRAVLJENO BESEDILO:", 'The corrected text:', 'The corrected text is:', 
                  'Corrected text:', 'Corrected text is:', "The correct spelling and grammar are:"]

In [26]:
def postprocess_segment_type(segment_type):
    def postprocess(row):
        pred_text = row[f'prediction.{segment_type}']
        ground_text = row[f'groundtruth.{segment_type}']
        prompt_text = row['prompt_text']
        
        if pred_text is not None:
            if type(pred_text) == str:
                if len(pred_text.strip()) > 0:
                    if pred_text.startswith('"'):
                        pred_text = pred_text[1:]
                    if pred_text.endswith('"'):
                        pred_text = pred_text[:-1]
                
                empty_prompt_text = prompt_text.replace('{{TEXT}}', '').strip()
#                 print('PROMP:', empty_prompt_text)
                
                if prompt_text in pred_text:
                    pred_text = pred_text.replace(prompt_text, '').strip()
                prompt_text_empty = prompt_text.replace("{{TEXT}}", '').strip()
                if prompt_text_empty in pred_text:
                    pred_text = pred_text.replace(prompt_text_empty, '').strip()
                prompt_text = prompt_text.replace("{{TEXT}}", pred_text)
                if prompt_text in pred_text:
                    pred_text = pred_text.replace(prompt_text, '').strip()
                pred_text = pred_text.strip()
                
                
                if empty_prompt_text in pred_text:
                    pred_text = pred_text.replace(empty_prompt_text, '')
#                     print('Prompt replaced.')

                for text in texts_to_remove:
                    if text in pred_text:
                        pred_text = pred_text[pred_text.index(text)+len(text):].strip()
#                         print(ground_text[:40], '----', pred_text[:40], '....')
#                         print('--'*20)


        return pred_text
    return postprocess

In [27]:
# !pip install git+https://github.com/google-research/bleurt.git

In [28]:
# pip install sacrebleu

In [29]:
# !pip install datasets

In [30]:
# !pip install sacrebleu

In [31]:
from datasets import load_metric

bleurt = load_metric("sacrebleu", cache_dir='cache')

bleurt.compute(references=[["sample text"]], predictions=["sample text"])

/tmp/ipykernel_1589662/3664822958.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleurt = load_metric("sacrebleu", cache_dir='cache')


{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 2,
 'ref_len': 2}

In [32]:
bleurt.compute(references=[["sample text"]], predictions=["sample text hjgghj"])

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [3, 2, 1, 0],
 'precisions': [66.66666666666667, 50.0, 50.0, 0.0],
 'bp': 1.0,
 'sys_len': 3,
 'ref_len': 2}

In [33]:
results[idx]

NameError: name 'idx' is not defined

In [ ]:
import os

path = 'data/processed_data'
os.makedirs(path, exist_ok=True)


In [ ]:
for idx, result in tqdm(enumerate(results), total=len(results)):
    
    try:
        results[idx] = results[idx].fillna('No text')
    except:
        print(idx)
        pass
    
    dataset_name = results[idx]['dataset_name'].unique()[0]
    model_name = results[idx]['model'].unique()[0]
    prompt = results[idx]['prompt'].unique()[0]
    prompt_text = results[idx]['prompt_text'].unique()[0]
    run_type = results[idx]['type'].unique()[0]
    
    print('Dataset:', dataset_name, 'Model:', model_name, 'Prompt:', prompt, 'Prompt text:', prompt_text)
    
    if 'icdar' in dataset_name:
        text_types = ['sentence', 'region']
    else:
        text_types = ['line', 'sentence', 'region']
    for segment_type in text_types:
        #try:
            results[idx]['length'] = results[idx][f'groundtruth.{segment_type}'].str.len()
            results[idx] = results[idx][results[idx]['length'] > 3]

            postprocess = postprocess_segment_type(segment_type)
            results[idx][f'prediction.{segment_type}'] = results[idx].apply(postprocess, axis=1)

            results[idx][f'{segment_type}-lev-ocr'] = \
                results[idx].parallel_apply(lambda x: compute_normalized_levenshtein_similarity(x[f'groundtruth.{segment_type}'],
                                                                                     x[f'ocr.{segment_type}']), axis=1)
            
        
            results[idx][f'{segment_type}-lev-pred'] = \
                results[idx].parallel_apply(lambda x: compute_normalized_levenshtein_similarity(x[f'groundtruth.{segment_type}'],
                                                                                     x[f'prediction.{segment_type}']), axis=1)

            results[idx][f'{segment_type}-lev-improvement'] = \
                results[idx].parallel_apply(lambda x: get_improvement(x[f'{segment_type}-lev-ocr'],
                                                             x[f'{segment_type}-lev-pred']), axis=1)

#             import pdb;pdb.set_trace()
#         except Exception as ex:
#             print(ex)
            
    print(f'Saving to data/processed_data/{idx}_{run_type}_{prompt_text}_{dataset_name}_{model_name}.csv')
    results[idx].to_csv(f"data/processed_data/{idx}_{run_type}_{prompt_text}_{dataset_name}_{model_name}.csv") 

    

  0%|                                                                                                                                                                                                                                                             | 0/1113 [00:00<?, ?it/s]

Dataset: icdar-2019 Model: facebook-opt-6.7b Prompt: prompt_complex_lang Prompt text: prompt_complex_03_per_lang


  0%|▏                                                                                                                                                                                                                                                 | 1/1113 [01:02<19:09:33, 62.03s/it]

Saving to data/processed_data/0_zero-shot_prompt_complex_03_per_lang_icdar-2019_facebook-opt-6.7b.csv
Dataset: icdar-2019 Model: gpt-4 Prompt: prompt_complex_lang Prompt text: prompt_complex_03_per_lang


  0%|▍                                                                                                                                                                                                                                                 | 2/1113 [02:03<19:00:36, 61.60s/it]

Saving to data/processed_data/1_zero-shot_prompt_complex_03_per_lang_icdar-2019_gpt-4.csv
Dataset: icdar-2019 Model: gpt-3.5-turbo Prompt: prompt_complex_lang Prompt text: prompt_complex_03_per_lang


  0%|▋                                                                                                                                                                                                                                                 | 3/1113 [03:05<19:03:06, 61.79s/it]

Saving to data/processed_data/2_zero-shot_prompt_complex_03_per_lang_icdar-2019_gpt-3.5-turbo.csv
Dataset: icdar-2019 Model: davinci Prompt: prompt_complex_lang Prompt text: prompt_complex_03_per_lang


In [ ]:
from datasets import load_metric
bleurt = load_metric("sacrebleu", cache_dir='cache')


In [ ]:
results

## Step 2: Generating LEV similarities 

In [ ]:
len(results)


In [ ]:
import pandas as pd

## Step 3: Preparing the final results (results concatenation + generating quality bands, etc.)


In [ ]:
data = pd.concat(results)

len(data)

In [ ]:
#####
MODEL_MAP = {'gpt-4':'GPT-4', 
             'gpt-3.5-turbo':'GPT-3.5', 
             'facebook-opt-350m':'OPT-350M',
             'bigscience-bloom-560m':'BLOOM-560M', 
             'decapoda-research-llama-7b-hf':'LLAMA-7B',
             'davinci':'GPT-3', 'gpt2':'GPT-2', 
             'tloen-alpaca-lora-7b':'Alpaca', 
             '3few-shot-gpt-4': 'GPT-4', 
             '3few-shot-gpt-3.5-turbo': 'GPT-3.5', 
             '3few-shot-davinci': 'GPT-3',
             '3few-shot-gpt2': 'GPT-2', 
             '3few-shot-facebook-opt-350m': 'OPT-350M', 
             'ajmc_primary_text-gpt-4': 'GPT-4', 
             'ajmc_primary_text-bigscience-bloom-560m': 'BLOOM-560M', 
             'ajmc_primary_text-decapoda-research-llama-7b-hf': 'LLAMA-7B', 
             'ajmc_primary_text-davinci': 'GPT-3', 
             'ajmc_primary_text-facebook-opt-350m': "OPT-350M",
             'ajmc_primary_text-gpt2': 'GPT-2',
             'ajmc_primary_text-gpt-3.5-turbo': 'GPT-3.5', 
             'ajmc_mixed-decapoda-research-llama-7b-hf': 'LLAMA-7B',
             'ajmc_mixed-bigscience-bloom-560m': 'BLOOM-560M',
             'ajmc_mixed-gpt2': 'GPT-2',
             'ajmc_mixed-facebook-opt-350m': 'OPT-350M',
             'ajmc_mixed-gpt-4': 'GPT-4',
             'ajmc_mixed-davinci': 'GPT-3',
             'ajmc_mixed-gpt-3.5-turbo': 'GPT-3.5',
             '3few-shot-bigscience-bloom-560m': 'BLOOM-560M',
             '3few-shot-decapoda-research-llama-7b-hf': 'LLAMA-7B',
             '3few-shot-impresso-nzz-bigscience-bloom-560m': 'BLOOM-560M',
             '3few-shot-impresso-nzz-decapoda-research-llama-7b-hf': 'LLAMA-7B',
             '3few-shot-impresso-nzz-facebook-opt-350m': 'OPT-350M',
             '3few-shot-..-..-llama-v2-llama-llama-2-7b-': 'LLAMA-2-7B',
             '..-..-llama-v2-llama-llama-2-7b-': 'LLAMA-2-7B',
             'facebook-opt-6.7b': 'OPT-6.7B',
             'bigscience-bloom-3b': 'BLOOM-3B',
             'bigscience-bloom-7b1': 'BLOOM-7.1B',
             '3few-shot-bigscience-bloom-7b1': 'BLOOM-7.1B',
             'ina-facebook-opt-6.7b': 'OPT-6.7B',
             'decapoda-research-llama-13b-hf': 'LLAMA-13B',
             'bigscience-bloomz-3b': 'BLOOMZ-3B',
             'bigscience-bloomz-7b1': 'BLOOMZ-7.1B',
             'bigscience-bloomz-560m': 'BLOOMZ-560M',
             '3few-shot-bigscience-bloom-3b': 'BLOOM-3B',
             '3few-shot-facebook-opt-6.7b': 'OPT-6.7B',
             '3few-shot-bigscience-bloomz-560m': 'BLOOMZ-560M',
             '3few-shot-bigscience-bloomz-7b1': 'BLOOMZ-7.1B',
             '3few-shot-bigscience-bloomz-3b': 'BLOOMZ-3B',
             'meta-llama-Llama-2-7b-hf': 'LLAMA-2-7B',
             '3few-shot-meta-llama-Llama-2-7b-hf': 'LLAMA-2-7B',
             'decapoda-research-llama-65b-hf': 'LLAMA-65B',
             'gpt-3.5-turbo-OLD': 'GPT-3-OLD'
            }
data['model'] = data['model'].apply(lambda x: MODEL_MAP[x])



In [ ]:
data.model.unique()


In [ ]:
data.prompt.unique()


In [ ]:
# Define OCR noise level bins
bins = [0, 0.4, 0.6, 0.8, 0.99, 1.0]

# Assign OCR noise level labels
labels = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# Create a new column for the OCR noise level bins
data[f"Quality Band"] = pd.cut(data[f'region-lev-ocr'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
data[f"Quality Band"].unique()


In [ ]:
# Add Overall Levenshtein Improvement
data[f'Overall Levenshtein Improvement'] = data[[f'line-lev-improvement', 
                                                 f'sentence-lev-improvement', 
                                                 f'region-lev-improvement']].mean(axis=1)



In [ ]:
# Custom binning function
def bin_improvement(x):
    if x < 0:
        return "Negative Improvement"
    elif x == 0:
        return "No Improvement"
    elif x > 0:
        return "Positive Improvement"

# Apply the function
data['Improvement Band'] = data['Overall Levenshtein Improvement'].apply(bin_improvement)



# Sampling

## Sampling for few-shot (commented out)

In [ ]:
# import numpy as np
# import pandas as pd
# from tqdm import tqdm

# # Define five distinct quality bands
# quality_bands = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# # Initialize an empty list for samples
# all_samples = []

# folder_few_shot = '../data/prompts/few_shot/'

# # Iterate over all unique datasets
# for dataset in tqdm(['ina'], total=len(['ina'])):
# # for dataset in tqdm(data['dataset_name'].unique(), total=len(data['dataset_name'].unique())):
    
    
#     # Get the unique languages for the current dataset
#     languages = data[data['dataset_name'] == dataset]['language'].unique()
#     prompts = data[data['dataset_name'] == dataset]['prompt'].unique()
    
#     print(len(data[data['dataset_name']==dataset]))
#     # Iterate over each unique language
#     for language in languages:
#         # Iterate over each unique prompt
#         for prompt in prompts:
#             sample_list = []
            
#             output_folder = os.path.join(folder_few_shot, dataset)
#             if not os.path.exists(output_folder):
#                 os.makedirs(output_folder)
            
#             # Repeat the sampling process until we have 3 samples
#             while len(sample_list) < 3:
#                 # Iterate over each unique quality band
#                 for quality_band in quality_bands:
                    
#                     # Filter the data to only include rows that match the current dataset, language, prompt, and quality band
#                     subset = data[(data['dataset_name'] == dataset) 
#                                   & (data['language'] == language)
#                                   & (data['prompt'] == prompt)
#                                   & (data['Quality Band'] == quality_band)
#                                   & (data['prediction.prompt'] != 'No text')
#                                   & (data['groundtruth.sentence'].str.len() > 10)]
                    
#                     # If the subset is not empty and we need more samples, take a sample
#                     if not subset.empty and len(sample_list) < 3:
#                         sample = subset.sample(1, random_state=1)
# #                         print(sample)
#                         sample_list.append(sample)
#                 # Break the loop if we already have 3 samples
#                 if len(sample_list) >= 3:
#                     break

#             all_samples.extend(sample_list)
#             if 'icdar' in dataset_name:
#                 text_types = ['sentence', 'region']
#             else:
#                 text_types = ['line', 'sentence', 'region']
            
# #             Generating prompts
#             print(prompt)
#             for segment_type in text_types:
#                 output_file = os.path.join(output_folder, f'{prompt}_{segment_type}_{language}.txt')
#                 with open(output_file, 'w') as f:
#                     for sample in sample_list:
#                         sample = sample.iloc[0]
#                         prompt_text = sample['prompt_text'].replace('{{TEXT}}', sample[f'ocr.{segment_type}'])
#                         correct_text = sample[f'groundtruth.{segment_type}']
#                         f.write(f'{prompt_text}\n\n{correct_text}\n\n')
#                         f.write(sample['prompt_text'])

# # Concatenate all the samples into a single DataFrame
# sample_df = pd.concat(all_samples, ignore_index=True)



## Sampling for analysis

In [ ]:
# import numpy as np
# import pandas as pd
# from tqdm import tqdm

# dataset_names = data.dataset_name.unique()
# quality_bands = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# sample_list = []
# # Iterate over all unique datasets
# for dataset in tqdm(dataset_names, total = len(dataset_names)):
#     # Get the unique languages for the current dataset
#     languages = data[data['dataset_name'] == dataset]['language'].unique()
#     print(dataset)
#     # Iterate over each unique language
#     for language in languages:
#         print('  --', language)
#         groundtruth_samples = data[(data['dataset_name'] == dataset) 
#                                    & (data['language'] == language)
#                                    & (data['groundtruth.sentence'].str.len() > 10)].drop_duplicates(subset=['groundtruth.line', 'groundtruth.sentence', 'groundtruth.region'])
#         # Limit the groundtruth_samples to three
#         if len(groundtruth_samples) >= 3:
#             groundtruth_samples = groundtruth_samples.sample(3, random_state=1335)

            
#         print(len(groundtruth_samples))             
#         # Iterate over each unique groundtruth samples
#         for idx, gt_sample in groundtruth_samples.iterrows():
#             prompts = data[data['dataset_name'] == dataset]['prompt'].unique()
                           
#             models = data[data['dataset_name'] == dataset]['model'].unique()
                          
#             improvement_bands = data[data['dataset_name'] == dataset]['Improvement Band'].unique()
#             is_few_shot_or_not = data[data['dataset_name'] == dataset]['type'].unique()

#             # Iterate over each unique prompt
#             for prompt in prompts:
#                 print('    -', prompt)
#                 # Iterate over each unique model
#                 for model in models:
# #                     print('     --', model)
#                     # Iterate over each quality band
#                     for band in quality_bands:
# #                         print('        ---', band)
#                         # Iterate over each improvement band
#                         for improvement_band in improvement_bands:
# #                             print('          ----', improvement_band)
#                             for is_few_shot in is_few_shot_or_not:
# #                                 print('-------', is_few_shot)
#                                 subset = data[(data['dataset_name'] == dataset) 
#                                               & (data['language'] == language)
#                                               & (data['prompt'] == prompt)
#                                               & (data['model'] == model)
#                                               & (data['Improvement Band'] == improvement_band)
#                                               & (data['Quality Band'] == band)
#                                               & (data['type'] == is_few_shot)
#                                               & (data['groundtruth.line'] == gt_sample['groundtruth.line'])
#                                               & (data['groundtruth.sentence'] == gt_sample['groundtruth.sentence'])
#                                               & (data['groundtruth.region'] == gt_sample['groundtruth.region'])]
                                          
#                                 # If the subset is not empty, take a sample
#                                 if not subset.empty:
#                                     sample = subset.sample(1, random_state=1, replace=True)
#                                     sample_list.append(sample)
#     #                                 print(sample)
#     #                             else:
#     #                                 print(f"No samples for Dataset: {dataset}, Language: {language}")

                                      
# # Concatenate all the samples into a single DataFrame
# sample_df = pd.concat(sample_list, ignore_index=True)
                                      

### Order columns

In [ ]:
# sample_df = sample_df.drop(['length', 'NbAlignedChar', 'prompt_text', 'File'], axis=1)

In [ ]:

# # 'index', 
# order = ['filename', 'dataset_name', 'model', 'language', 'prompt', 
#          'Overall Levenshtein Improvement', 'Quality Band', 'Improvement Band',
#          'ocr.line', 'groundtruth.line', 'prediction.line', 
#          'line-lev-ocr', 'line-lev-pred', 'line-lev-improvement',
#          'ocr.sentence', 'groundtruth.sentence', 'prediction.sentence', 
#          'sentence-lev-ocr', 'sentence-lev-pred', 'sentence-lev-improvement', 
#          'ocr.region', 'groundtruth.region', 'prediction.region',
#          'region-lev-ocr', 'region-lev-pred', 'region-lev-improvement', 
#          'article_id', 'century', 'Date', 'Type']

# # Reorder the DataFrame
# sample_df = sample_df[order]


### Write sample

In [ ]:
# from datetime import datetime

# # Use today's date for the filename
# today = datetime.now().strftime('%d%B')  # This will format the date as 'DayMonth'

# # Save the DataFrame to a csv file
# sample_df.to_csv(f'ResultsGPTUpdated{today}.csv', index=False)


### Distribution of WER/CER rates for all datasets in the four quality bands, established via Levenshtein similarity.

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings('ignore')

# # Define the bins and labels for quality bands
# bins = [0, 0.7, 0.8, 0.9, 1]
# labels = ["0-70%", "70-80%", "80-90%", "90-100%"]

# # Count the number of unique datasets
# n_datasets = data.dataset_name.nunique()
# dataset_names = ['impresso-nzz', 'overproof', 'ajmc-mixed', 
#                  'ajmc-primary-text', 'icdar-2017', 'icdar-2019', 'htrec']

# for error_rate in ['cer', 'wer']:
#     # Create subplots
#     fig, axs = plt.subplots(3, 4, figsize=(20, 15))

#     # Flatten the axes for easy iteration
#     axs = axs.flatten()

#     for i, dataset in enumerate(dataset_names):
#         dataset_data = data[data.dataset_name == dataset]

#         # Compute the mean WER across line, sentence, and region levels
#         dataset_data[f'Mean {error_rate.upper()}'] = dataset_data[[f'line-{error_rate}-ocr', 
#                                                  f'sentence-{error_rate}-ocr', 
#                                                  f'region-{error_rate}-ocr']].mean(axis=1)

#         # Plot the distribution of WERs for each quality band
#         for band in labels:
#             band_df = dataset_data[dataset_data[f"{segment_type}-ocr-noise-group"] == band]

#             _ = sns.histplot(band_df, x=f"Mean {error_rate.upper()}", 
#                              label=f"Quality Band {band}", kde=True, ax=axs[i])

#         axs[i].set_xlim([0, 100])
#         axs[i].set_title(f'{dataset.upper()}')
#         axs[i].legend()

#     # Remove empty subplots
#     for i in range(len(data.dataset_name.unique()), len(axs)):
#         _ = fig.delaxes(axs[i])

#     plt.tight_layout()
#     plt.suptitle(f'Mean {error_rate.upper()} Error Rates across Datasets and Quality Bands', fontsize=20, y=1.02)
#     plt.show()

In [ ]:
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt
# import warnings
# warnings.filterwarnings('ignore')

# # Define OCR noise level bins
# # bins = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
# # bins = [0, 0.7, 0.8, 0.9, 1.0]
# bins = [0, 0.4, 0.6, 0.8, 0.99, 1.0]

# # Assign OCR noise level labels
# # labels = ["0-10%", "10-20%", "20-30%", "30-40%", "40-50%", "50-60%", "60-70%", "70-80%", "80-90%", "90-100%"]
# # labels = ["0-70%", "70-80%", "80-90%", "90-100%"]
# labels = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# # Count the number of unique datasets
# n_datasets = data.dataset_name.nunique()
# dataset_names = ['impresso-nzz', 'overproof', 'ajmc-mixed', 
#                  'ajmc-primary-text', 'icdar-2017', 'icdar-2019', 'htrec']

# for error_rate in ['lev']:
#     # Create subplots
#     fig, axs = plt.subplots(3, 4, figsize=(20, 15))

#     # Flatten the axes for easy iteration
#     axs = axs.flatten()

#     for i, dataset in enumerate(dataset_names):
#         dataset_data = data[data.dataset_name == dataset]

#         # Compute the mean WER across line, sentence, and region levels
#         dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
#                                                           f'sentence-{error_rate}-improvement', 
#                                                           f'region-{error_rate}-improvement']].mean(axis=1)
#         # Plot the distribution of WERs for each quality band
#         for band in labels:
#             band_df = dataset_data[dataset_data[f"{segment_type}-ocr-noise-group"] == band]

#             _ = sns.histplot(band_df, x=f"Overall Levenshtein Improvement", 
#                              label=f"Quality Band {band}", kde=True, ax=axs[i])

# #         axs[i].set_ylim([0, 300])
#         axs[i].set_xlim([-1, 1])
#         axs[i].set_title(f'{dataset.upper()}')
#         axs[i].legend()

#     # Remove empty subplots
#     for i in range(len(data.dataset_name.unique()), len(axs)):
#         fig.delaxes(axs[i])

#     plt.tight_layout()
#     plt.suptitle('Overall Levenshtein Improvement across Datasets and Quality Bands', fontsize=20, y=1.02)
#     plt.show()

In [ ]:
data.type.unique()


In [ ]:
data.prompt.unique()


In [ ]:
data['Improvement Band'].unique()


In [ ]:
data.model.unique()


In [ ]:
data = data.loc[data['model'] != 'LLaMA-13B']
data = data.loc[data['model'] != 'LLAMA-65B']
data = data.loc[data['model'] != 'GPT-3-OLD']




In [ ]:
data.model.unique()


In [ ]:
MODELS = [
       'LLAMA-7B', 'LLAMA-2-7B', 
       'BLOOM-560M',  'BLOOM-3B',  'BLOOM-7.1B', 
       'BLOOMZ-560M', 'BLOOMZ-3B', 'BLOOMZ-7.1B', 
       'OPT-350M', 'OPT-6.7B',
       'GPT-2', 'GPT-3', 'GPT-3.5', 'GPT-4']
MODELS

In [ ]:
limited_models = ['LLAMA-7B', 'LLAMA-2-7B', 
       'BLOOM-560M',  'BLOOM-3B',  'BLOOM-7.1B', 
       'BLOOMZ-560M', 'BLOOMZ-3B', 'BLOOMZ-7.1B', 
       'OPT-350M', 'OPT-6.7B',
       'GPT-2']  # Replace these with your list of 'limited' models
open_models = ['GPT-3', 'GPT-3.5', 'GPT-4']  # Replace these with your list of 'open' models

# Create new 'Access' column
data['Access'] = data['model'].apply(lambda x: 'limited' if x in limited_models else ('open' if x in open_models else 'unknown'))


In [ ]:
data.columns

# Plots

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.8)
sns.set_style("whitegrid")

# Define OCR noise level bins
bins = [0, 0.4, 0.6, 0.8, 0.99, 1.0]

# Assign OCR noise level labels
labels = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# Count the number of unique datasets
n_datasets = data.dataset_name.nunique()
dataset_names = ['impresso-nzz', 'overproof', 'ajmc-mixed', 
                 'ajmc-primary-text', 'icdar-2017', 'icdar-2019', 'htrec', 'ina']

prompt_names = ['prompt_basic_01', 'prompt_basic_02', 'prompt_complex_01', 
                'prompt_complex_02', 'prompt_complex_lang']


n_plots = len(dataset_names)
n_plots_per_figure = 4
n_figures = int(np.ceil(n_plots / n_plots_per_figure))

print(n_figures)

for type_of_experiment in ['zero-shot', 'few-shot']:
# for type_of_experiment in ['language-specific']:
    for error_rate in ['lev']:
        
        for fig_idx in range(n_figures):
            fig, axs = plt.subplots(2, 2, figsize=(30, 15))
            axs = axs.flatten()

            for i in range(n_plots_per_figure):
                idx = fig_idx * n_plots_per_figure + i
                if idx < n_plots:
                    dataset = dataset_names[idx]
                dataset_data = data[(data.dataset_name == dataset) & (data.type == type_of_experiment)]
                # Compute the mean WER across line, sentence, and region levels
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                      f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)

                try:
                    # Plot the distribution of improvements for each model
                    _ = sns.boxplot(x='model', y=f'Overall Levenshtein Improvement', data=dataset_data, 
                                    ax=axs[i], order=MODELS, hue='prompt', hue_order=prompt_names)
                    axs[i].set_title(f'{dataset.upper()} ({type_of_experiment})')

                    axs[i].set_ylim([-1, 1.2])
                    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=15)  # Rotate x-axis labels
                    axs[i].set_xlabel('')  # Remove x-axis label
                    axs[i].set_ylabel('')  # Remove y-axis label
                except Exception as ex:
                    print(f'Could not load {dataset} with {ex}')


            # Remove empty subplots
            for i in range(len(data.dataset_name.unique()), len(axs)):
                fig.delaxes(axs[i])

            plt.tight_layout()
            plt.suptitle(f'Overall Levenshtein Improvement across Datasets, Models, and Prompts ({type_of_experiment})', fontsize=20, y=1.02)
            plt.show()


In [ ]:
sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.)
sns.set_style("whitegrid")

prompt_names = ['prompt_basic_01', 'prompt_basic_02', 'prompt_complex_01', 
                'prompt_complex_02', 'prompt_complex_lang']


for type_of_experiment in ['zero-shot', 'few-shot']:#
    for error_rate in ['lev']:
        for fig_idx in range(n_figures):
            fig, axs = plt.subplots(2, 2, figsize=(17, 9))
            axs = axs.flatten()

            for i in range(n_plots_per_figure):
                idx = fig_idx * n_plots_per_figure + i
                if idx < n_plots:
                    dataset = dataset_names[idx]
                    
                dataset_data = data[(data.dataset_name == dataset) & (data.type == type_of_experiment)]
                
                dataset_data['model'] = pd.Categorical(dataset_data['model'], categories=MODELS, ordered=True)
                
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                      f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)

                try:
                    # Plot the line plot of improvements for each model
                    _ = sns.lineplot(x='model', y=f'Overall Levenshtein Improvement', 
                                     data=dataset_data.sort_values('model'), 
                                    ax=axs[i], hue='prompt', sort=False, linewidth=2)
                    axs[i].set_title(f'{dataset.upper()} ({type_of_experiment})')
                    axs[i].set_ylim([-1, 1.2])
                    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=15)
                    axs[i].set_xlabel('Prompt Complexity')  
                    axs[i].set_ylabel('Overall Levenshtein Improvement')  
                except Exception as ex:
                    print(f'Could not load {dataset} with {ex}')

            # Remove empty subplots
            for i in range(len(data.dataset_name.unique()), len(axs)):
                fig.delaxes(axs[i])

            plt.tight_layout()
            plt.suptitle(f'Evolution of Prompt Complexity across Datasets, Models, and Improvements ({type_of_experiment})', fontsize=20, y=1.02)
            plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1)
sns.set_style("whitegrid")

for type_of_experiment in ['zero-shot', 'few-shot']:
    for error_rate in ['lev']:
        # Create a new DataFrame to store the average improvements for each language and prompt
        average_improvements = pd.DataFrame(columns=['prompt', 'language', f'Overall Levenshtein Improvement'])

        for language in data.language.unique():  # Iterate over each unique language
            for dataset in dataset_names:
                dataset_data = data[(data.dataset_name == dataset) & (data.language == language) & (data.type == type_of_experiment)]  # Filter data for the current language
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)

                # Group the data by prompt and language, then calculate the mean Overall Levenshtein Improvement
                grouped_data = dataset_data.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

                # Add the grouped data to the average_improvements DataFrame
                average_improvements = pd.concat([average_improvements, grouped_data])

        # Group the data by prompt and language again, this time averaging the averages for each language and prompt
        average_improvements = average_improvements.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

        fig, ax = plt.subplots(figsize=(10, 5))  # Create the plot

        try:
            # Plot the line plot of average improvements for each prompt
            _ = sns.lineplot(x='prompt', y=f'Overall Levenshtein Improvement', hue='language', data=average_improvements, 
                             ax=ax, legend='full')  # Add hue='language' to differentiate lines by language
        except Exception as ex:
            print(f'Could not load data with {ex}')

        ax.set_title(f'Average Overall Levenshtein Improvement ({type_of_experiment})')  # Modify title since it's no longer specific to one language
        ax.set_ylim([-1, 1.2])
        ax.set_xlabel('Prompt Complexity')  
        ax.set_ylabel('Average Overall Levenshtein Improvement')

        plt.tight_layout()
        plt.suptitle(f'Evolution of Prompt Complexity and Average Improvement ({type_of_experiment})', fontsize=20, y=1.02)
        plt.show()



In [ ]:
for type_of_experiment in ['zero-shot', 'few-shot']:
    for error_rate in ['lev']:
        # Create a new DataFrame to store the average improvements for each language and prompt
        average_improvements = pd.DataFrame(columns=['prompt', 'language', f'Overall Levenshtein Improvement'])

        for language in data.language.unique():  # Iterate over each unique language
            for dataset in dataset_names:
                dataset_data = data[(data.dataset_name == dataset) & (data.language == language) & (data.type == type_of_experiment)]  # Filter data for the current language
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)

                # Group the data by prompt and language, then calculate the mean Overall Levenshtein Improvement
                grouped_data = dataset_data.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

                # Add the grouped data to the average_improvements DataFrame
                average_improvements = pd.concat([average_improvements, grouped_data])

        # Group the data by prompt and language again, this time averaging the averages for each language and prompt
        average_improvements = average_improvements.groupby(['prompt', 'language'])[f'Overall Levenshtein Improvement'].mean().reset_index()

        fig, ax = plt.subplots(figsize=(15, 10))  # Create the plot

        try:
            # Plot the KDE plot of average improvements for each prompt
            for language in average_improvements.language.unique():
                _ = sns.kdeplot(average_improvements[average_improvements.language == language][f'Overall Levenshtein Improvement'], 
                                ax=ax, label=language, lw=2.5)  # Increase line width here
        except Exception as ex:
            print(f'Could not load data with {ex}')

        ax.set_title(f'Average Overall Levenshtein Improvement KDE ({type_of_experiment})')  # Modify title since it's no longer specific to one language
        ax.set_xlabel('Average Overall Levenshtein Improvement')
        ax.set_ylabel('Density')

        plt.tight_layout()
        plt.legend(title='Language', title_fontsize='13', fontsize='12')  # Increase legend fontsize here
        plt.suptitle(f'Evolution of Prompt Complexity and Average Improvement KDE ({type_of_experiment})', fontsize=20, y=1.02)
        plt.show()


In [ ]:
data.columns

In [ ]:
for type_of_experiment in ['zero-shot', 'few-shot']:
    for error_rate in ['lev']:
        # Create a new DataFrame to store the average improvements for each model and prompt
        average_improvements = pd.DataFrame(columns=['prompt', 'model', f'Overall Levenshtein Improvement'])

        for model in data.model.unique():  # Iterate over each unique model
            for dataset in dataset_names:
                dataset_data = data[(data.dataset_name == dataset) & (data.model == model) & (data.type == type_of_experiment)]  # Filter data for the current model
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)

                # Group the data by prompt and model, then calculate the mean Overall Levenshtein Improvement
                grouped_data = dataset_data.groupby(['prompt', 'model'])[f'Overall Levenshtein Improvement'].mean().reset_index()

                # Add the grouped data to the average_improvements DataFrame
                average_improvements = pd.concat([average_improvements, grouped_data])

        # Group the data by prompt and model again, this time averaging the averages for each model and prompt
        average_improvements = average_improvements.groupby(['prompt', 'model'])[f'Overall Levenshtein Improvement'].mean().reset_index()

        fig, ax = plt.subplots(figsize=(15, 10))  # Create the plot

        try:
            # Plot the KDE plot of average improvements for each prompt
            for model in average_improvements.model.unique():
                _ = sns.kdeplot(average_improvements[average_improvements.model == model][f'Overall Levenshtein Improvement'], 
                                ax=ax, label=model, lw=2.5)  # Increase line width here
        except Exception as ex:
            print(f'Could not load data with {ex}')

        ax.set_title(f'Average Overall Levenshtein Improvement KDE ({type_of_experiment})')  # Modify title since it's no longer specific to one model
        ax.set_xlabel('Average Overall Levenshtein Improvement')
        ax.set_ylabel('Density')

        plt.tight_layout()
        plt.legend(title='Model', title_fontsize='13', fontsize='12')  # Increase legend fontsize here
        plt.suptitle(f'Evolution of Prompt Complexity and Average Improvement KDE ({type_of_experiment})', fontsize=20, y=1.02)
        plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import numpy as np
warnings.filterwarnings('ignore')

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.5)
sns.set_style("whitegrid")

# Define OCR noise level bins
bins = [0, 0.4, 0.6, 0.8, 0.99, 1.0]

# Assign OCR noise level labels
labels = ["0-40%", "40-60%", "60-80%", "80-99%", "99-100%"]

# Count the number of unique datasets
n_datasets = data.dataset_name.nunique()
dataset_names = ['impresso-nzz', 'overproof', 'ajmc-mixed', 
                 'ajmc-primary-text', 'icdar-2017', 'icdar-2019', 'htrec', 'ina']

prompt_names = ['prompt_basic_01', 'prompt_basic_02', 'prompt_complex_01', 
                'prompt_complex_02', 'prompt_complex_lang']


n_plots = len(dataset_names)
n_plots_per_figure = 4
n_figures = int(np.ceil(n_plots / n_plots_per_figure))

for type_of_experiment in ['zero-shot', 'few-shot']:
# for type_of_experiment in ['language-specific']:
    for error_rate in ['lev']:
        
        for fig_idx in range(n_figures):
            fig, axs = plt.subplots(2, 2, figsize=(30, 15))
            axs = axs.flatten()

            for i in range(n_plots_per_figure):
                idx = fig_idx * n_plots_per_figure + i
                if idx < n_plots:
                    dataset = dataset_names[idx]
                dataset_data = data[(data.dataset_name == dataset) & (data.type == type_of_experiment)]
                # Compute the mean WER across line, sentence, and region levels
                if 'icdar' not in dataset:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement']].mean(axis=1)
                else:
                    dataset_data[f'Overall Levenshtein Improvement'] = dataset_data[[f'sentence-{error_rate}-improvement']].mean(axis=1)

                try:
                    # Plot the distribution of improvements for each model
                    _ = sns.boxplot(x='model', y=f'Overall Levenshtein Improvement', data=dataset_data, 
                                    ax=axs[i], order=MODELS, hue='prompt', hue_order=prompt_names)
                    axs[i].set_title(f'{dataset.upper()} ({type_of_experiment})')

                    axs[i].set_ylim([-1.1, 1.1])
                    axs[i].set_xticklabels(axs[i].get_xticklabels(), rotation=15)  # Rotate x-axis labels
                    axs[i].set_xlabel('')  # Remove x-axis label
                    axs[i].set_ylabel('')  # Remove y-axis label
                except Exception as ex:
                    print(f'Could not load {dataset} with {ex}')


            # Remove empty subplots
            for i in range(len(data.dataset_name.unique()), len(axs)):
                fig.delaxes(axs[i])

            plt.tight_layout()
            plt.suptitle(f'Overall Levenshtein Improvement across Datasets, Models, and Prompts ({type_of_experiment})', fontsize=20, y=1.02)
            plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=.7)
sns.set_style("whitegrid")

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        fig, axs = plt.subplots(3, 2, figsize=(8, 12))  # Change here
        axs = axs.flatten()  # To make it easy to index

        for i, prompt in enumerate(prompt_names):
            prompt_data = data[(data.prompt == prompt) & (data.type == type_of_experiment)]
            
            if 'icdar' not in dataset:
                prompt_data[f'Overall Levenshtein Improvement'] = prompt_data[[f'line-{error_rate}-improvement', 
                                                                  f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)
            else:
                prompt_data[f'Overall Levenshtein Improvement'] = prompt_data[[f'sentence-{error_rate}-improvement', 
                                                              f'region-{error_rate}-improvement']].mean(axis=1)

            try:
                if len(prompt_data) > 0:
                    sns.kdeplot(data=prompt_data, x=f'Overall Levenshtein Improvement', hue='model', 
                                fill=True, ax=axs[i], hue_order=MODELS)
                    axs[i].set_title(f'{prompt} ({type_of_experiment})')
                    axs[i].set_xlim([-1, 1.2])
                    axs[i].set_ylim([0, 1.4])
            except Exception as ex:
                print(f'Could not plot {prompt} with {ex}')

        # Remove empty subplot
        fig.delaxes(axs[-1])  # Change here

        plt.tight_layout()
        plt.show()
        
# A kernel density estimate (KDE) plot is a method for visualizing the distribution of observations in a dataset, 
# analogous to a histogram. KDE represents the data using a continuous probability density curve in one or more 
# dimensions.

# Relative to a histogram, KDE can produce a plot that is less cluttered and more interpretable, especially when 
# drawing multiple distributions. But it has the potential to introduce distortions if the underlying distribution 
# is bounded or not smooth. Like a histogram, the quality of the representation also depends on the selection of 
# good smoothing parameters.


In [ ]:
data.columns

In [ ]:
prompts_to_compare = ['prompt_complex_02', 'prompt_complex_lang']
languages = [lang for lang in data['language'].unique() if lang != 'en']  # Exclude 'en' language

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        plt.figure(figsize=(10, 6))  # Adjust as necessary

        for language in languages:
            language_data = data[(data.language == language) & 
                                 (data.type == type_of_experiment) & 
                                 (data.prompt.isin(prompts_to_compare))]

            if 'icdar' not in dataset:
                language_data[f'Overall Levenshtein Improvement'] = language_data[[f'line-{error_rate}-improvement', 
                                                                  f'sentence-{error_rate}-improvement', 
                                                                  f'region-{error_rate}-improvement']].mean(axis=1)
            else:
                language_data[f'Overall Levenshtein Improvement'] = language_data[[f'sentence-{error_rate}-improvement', 
                                                              f'region-{error_rate}-improvement']].mean(axis=1)

            try:
                if len(language_data) > 0:
                    sns.kdeplot(data=language_data, x=f'Overall Levenshtein Improvement', hue='prompt', 
                                fill=True, hue_order=prompts_to_compare)
            except Exception as ex:
                print(f'Could not plot {language} with {ex}')

        plt.title(f'All Languages ({type_of_experiment})')
        plt.tight_layout()
        plt.show()


In [ ]:
data.columns

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=1.)


prompts_to_compare = ['prompt_complex_02', 'prompt_complex_lang']
languages = [lang for lang in data['language'].unique() if lang != 'en']  # Exclude 'en' language

bar_data = []

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        for language in languages:
            for prompt in prompts_to_compare:
                sub_data = data[(data.language == language) & 
                                 (data.type == type_of_experiment) & 
                                 (data.prompt == prompt)]
                if 'icdar' not in dataset:
                    sub_data[f'Overall Levenshtein Improvement'] = sub_data[[f'line-{error_rate}-improvement', 
                                                                          f'sentence-{error_rate}-improvement', 
                                                                          f'region-{error_rate}-improvement']].mean(axis=1)
                else:
                    sub_data[f'Overall Levenshtein Improvement'] = sub_data[[f'sentence-{error_rate}-improvement', 
                                                                      f'region-{error_rate}-improvement']].mean(axis=1)
                
                mean_improvement = np.mean(sub_data[f'Overall Levenshtein Improvement'])
                bar_data.append({'Language': language, 'Prompt': prompt, 'Mean Improvement': mean_improvement})

bar_data = pd.DataFrame(bar_data)

plt.figure(figsize=(10, 6))
sns.barplot(x='Language', y='Mean Improvement', hue='Prompt', data=bar_data, hue_order=prompts_to_compare)
plt.title(f'Mean Levenshtein Improvement for all Languages ({type_of_experiment})')
plt.xticks(rotation=90)  # Rotate x-axis labels for better visibility
plt.tight_layout()
plt.show()


In [ ]:
data.columns

In [ ]:
data.dataset_name.unique()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_palette('colorblind')
sns.set_context("notebook", font_scale=.7)

segmentations = ['line', 'sentence', 'region']

for type_of_experiment in ['zero-shot']:
    for error_rate in ['lev']:
        for segmentation in segmentations:
            fig, axs = plt.subplots(3, 2, figsize=(8, 12))  # Change here
            axs = axs.flatten()  # To make it easy to index

            for i, prompt in enumerate(prompt_names):
                prompt_data = data[(data.prompt == prompt) & (data.type == type_of_experiment)]
                if segmentation == 'line':
                    prompt_data = prompt_data[~prompt_data.dataset_name.isin(['icdar-2017', 'icdar-2019'])]
                prompt_data[f'{segmentation.capitalize()} Levenshtein Improvement'] = prompt_data[f'{segmentation}-{error_rate}-improvement']

                try:
                    if len(prompt_data) > 0:
                        sns.kdeplot(data=prompt_data, x=f'{segmentation.capitalize()} Levenshtein Improvement', 
                                    hue='model', fill=True, ax=axs[i], hue_order=MODELS)
                        axs[i].set_title(f'{prompt} ({type_of_experiment})')
                        axs[i].set_xlim([-1, 1.2])
                        axs[i].set_ylim([0, 1.5])
                except Exception as ex:
                    print(f'Could not plot {prompt} with {ex}')

            # Remove empty subplot
            fig.delaxes(axs[-1])  # Change here

            plt.suptitle(f'{segmentation.capitalize()} Levenshtein Improvement across Prompts and Models ({type_of_experiment})', fontsize=20, y=1.02)
            plt.tight_layout()
            plt.show()


In [ ]:
data.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

results = []  # Define results as a list


for type_of_experiment in ['zero-shot', 'few-shot']:
    for model in MODELS:
        model_data = data[(data.model == model) & (data.type == type_of_experiment)]
        
        # Compute the mean Levenshtein Improvement across line, sentence, and region levels
        if 'icdar' not in dataset:
            model_data['Overall Levenshtein Improvement'] = model_data[[f'line-lev-improvement', 
                                                                        f'sentence-lev-improvement', 
                                                                        f'region-lev-improvement']].mean(axis=1)
        else:
            model_data['Overall Levenshtein Improvement'] = model_data[[f'sentence-lev-improvement', 
                                                                        f'region-lev-improvement']].mean(axis=1)

        # Append the results
#         print(model_data['Improvement Band'].unique())
        results.append({'Model': model,
                        'Type of Experiment': type_of_experiment,
                        'Overall Levenshtein Improvement': np.nanmean(model_data['Overall Levenshtein Improvement'])})

# Convert the results list to a DataFrame for plotting
results_df = pd.DataFrame(results)

# Create the plot
plt.figure(figsize=(10,6))
sns.barplot(x='Model', y='Overall Levenshtein Improvement', hue='Type of Experiment', data=results_df, order=MODELS)
plt.title('Overall Levenshtein Improvement for Zero-Shot and Few-Shot Models')
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


for type_of_experiment in ['zero-shot', 'few-shot']:
    results = []  # Define results as a list

    for model in MODELS:
        for band in data['Improvement Band'].unique():
            model_band_data = data[(data.model == model) & (data.type == type_of_experiment) & (data['Improvement Band'] == band)]
            
            # Compute the mean Levenshtein Improvement across line, sentence, and region levels
            if 'icdar' not in dataset:
                model_band_data['Overall Levenshtein Improvement'] = model_band_data[[f'line-lev-improvement', 
                                                                                    f'sentence-lev-improvement', 
                                                                                    f'region-lev-improvement']].mean(axis=1)
            else:
                model_band_data['Overall Levenshtein Improvement'] = model_band_data[[f'sentence-lev-improvement', 
                                                                                    f'region-lev-improvement']].mean(axis=1)

            # Append the results
            results.append({'Model': model,
                            'Type of Experiment': type_of_experiment,
                            'Overall Levenshtein Improvement': np.nanmean(model_band_data['Overall Levenshtein Improvement']),
                            'Improvement Band': band})

    # Convert the results list to a DataFrame for plotting
    results_df = pd.DataFrame(results)

    # Create the plot
    plt.figure(figsize=(10,6))
    sns.barplot(x='Model', y='Overall Levenshtein Improvement', hue='Improvement Band', 
                data=results_df, order=MODELS)
    plt.title('Overall Levenshtein Improvement for Zero-Shot and Few-Shot Models')
    plt.show()


In [ ]:
data.columns

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


g = sns.FacetGrid(data, col='Type', hue='Improvement Band', height=10, aspect=1)
g.map(sns.barplot, 'model', 'Overall Levenshtein Improvement', order=MODELS)

# Calculate means for each type of experiment and add horizontal lines
for ax, (type_of_experiment, item) in zip(g.axes.flatten(), data.groupby('Type')):
    mean_improvement = item['Overall Levenshtein Improvement'].mean()
    ax.axhline(mean_improvement, color='black', linestyle='--')
    ax.text(0.6, mean_improvement, f'Mean: {mean_improvement:.2f}', color='black')

plt.show()
